In [3]:
import io
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FileUpload, SelectMultiple, VBox, Button, Output, HBox, Dropdown, Layout
from IPython.display import display

# Widgets
upload_btn = FileUpload(accept='.txt,.csv,.xy', multiple=True)
dataset_selector = SelectMultiple(description='Datasets:', layout=Layout(width='300px', height='150px'))
plot_btn = Button(description='Plot', button_style='success')
export_btn = Button(description='Export', button_style='info')
export_format = Dropdown(options=['png', 'jpeg'], value='png', description='Format:')
output = Output()

# Data storage
uploaded_files = {}
current_fig = None

# File parser
def parse_file(file_content):
    try:
        text = file_content.decode('utf-8', errors='ignore')
        # Remove comment lines and empty lines
        lines = [line for line in text.splitlines() if line.strip() and not line.strip().startswith('#')]
        text_clean = '\n'.join(lines)
        
        # Try common delimiters
        for delim in [',', '\t', ' ']:
            try:
                df = pd.read_csv(io.StringIO(text_clean), sep=delim, header=None, engine='python')
                if df.shape[1] >= 2:
                    df = df.iloc[:, :2]
                    df.columns = ['x', 'y']
                    df = df.dropna()
                    return df
            except Exception:
                continue
    except Exception as e:
        print(f"Exception during parsing: {e}")
    return None

# Upload handler
def on_upload_change(change):
    uploaded_files.clear()
    dataset_selector.options = []
    files = upload_btn.value
    output.clear_output()

    # Handle both dict (newer) and list (older) formats
    if isinstance(files, dict):  # newer ipywidgets
        file_items = files.items()
    else:
        file_items = [(f['name'], f) for f in files]

    options = []
    for fname, file in file_items:
        content = file['content']
        df = parse_file(content)
        if df is not None:
            uploaded_files[fname] = df
            options.append(fname)
        else:
            with output:
                print(f"❌ Failed to parse: {fname}")

    dataset_selector.options = options

upload_btn.observe(on_upload_change, names='value')

# Plot
def on_plot_clicked(b):
    global current_fig
    output.clear_output()
    selected = dataset_selector.value
    if not selected:
        with output:
            print("⚠️ No datasets selected.")
        return

    fig, ax = plt.subplots(figsize=(8, 6))
    offset = 0
    for fname in selected:
        df = uploaded_files[fname]
        ax.plot(df['x'], df['y'] + offset, label=fname)
        offset += df['y'].max() * 1.2

    ax.set_xlabel('X')
    ax.set_ylabel('Y (stacked)')
    ax.set_title('Stacked Plot')
    ax.legend()
    fig.tight_layout()

    current_fig = fig
    with output:
        plt.show()

plot_btn.on_click(on_plot_clicked)

# Export
def on_export_clicked(b):
    if current_fig is None:
        with output:
            print("⚠️ Please generate a plot first.")
        return
    fmt = export_format.value
    filename = f'stacked_plot.{fmt}'
    current_fig.savefig(filename, format=fmt)
    with output:
        print(f"✅ Plot saved as: {filename}")

export_btn.on_click(on_export_clicked)

# Layout
ui = VBox([
    upload_btn,
    dataset_selector,
    HBox([plot_btn, export_btn, export_format]),
    output
])

display(ui)
